In [9]:
# By Maryam Rezayati
# Ref  for lstm: https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/

# this model should be trained in the same conda environment which robot will be runned
# conda activate frankapyenv

import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics import ConfusionMatrix
from torchvision import transforms
from sklearn.metrics import precision_score, recall_score, f1_score,  classification_report
from load_dataset import create_tensor_dataset_without_torque



AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

# load functions

In [2]:
# from main.py import get_output or
def get_output(data_ds, model):
    labels_pred = []
    model.eval()
    with torch.no_grad():
        for i in range(data_ds.data_target.shape[0]):
            x , y = data_ds.__getitem__(i)
            x = x[None, :]

            x = model(x)
            x = x.squeeze()
            #labels_pred.append(torch.Tensor.cpu(x.detach()).numpy())
            labels_pred.append(x.detach().numpy())
    #convert list type to array
    labels_pred = np.array(labels_pred)
    labels_pred = labels_pred.argmax(axis=1)
    labels_true = np.array(data_ds.data_target[:])
    labels_true = labels_true.astype('int64')

    return torch.tensor(labels_pred), torch.tensor(labels_true)

In [3]:
# from main.py import Sequence or
class Sequence(nn.Module):
    def __init__(self, num_class = 5, network_type='main',num_features_lstm=4):
        super(Sequence, self).__init__()
        hidden_size = 50
        self.lstm = nn.LSTM(input_size = num_features_lstm*28, hidden_size= hidden_size, num_layers= 1, batch_first = True)
        self.network_type = network_type
        if self.network_type == 'main':
            self.linear = nn.Linear(hidden_size, num_class)    
        else:
            self.linear = nn.Linear(hidden_size*7, num_class)

        #self.linear2 = nn.Linear(50, num_class)

    def forward(self, input, future = 0):
        x, _ = self.lstm(input)

        if self.network_type == 'main':
            x = x[:,-1,:]
        else:
            x = torch.flatten(x, start_dim=1)

        x = self.linear(x)
        #x = self.linear2(x)
        #x = x[:,-1,:]
        #print(x.shape)
        return x

In [5]:
def import_lstm_models(PATH:str, num_features_lstm=4):

	checkpoint = torch.load(PATH)

	model = Sequence(num_class = checkpoint["num_classes"], network_type = checkpoint["network_type"], num_features_lstm = num_features_lstm)
	model.load_state_dict(checkpoint["model_state_dict"])

	if checkpoint["collision"]:
		labels_map = { # , is for saving data
					0: ',Collaborative_Contact',
					1: ',Collision,'
						}
		print('collision detection model is loaded!')
		
	elif checkpoint["localization"]:
		labels_map = { # , is for saving data
					0: ',Link 5',
					1: ',Link 6,'
						}
		print('localization model is loaded!')
	
	elif checkpoint["num_classes"] == 5:
		labels_map = { # , is for saving data
					0: ',Noncontact,',
					1: ',Intentional_Link5,',
					2: ',Intentional_Link6,',
					3: ',Collision_Link5,',
					4: ',Collision_Link6,',
						}
		print('5-classes model is loaded!')

	elif checkpoint["num_classes"] == 3: 
		labels_map = { # , is for saving data
					0: ',Noncontact,',
					1: ',Collaborative_Contact,',
					2: ',Collision,',
				} 
		print('collision detection with 3 classes model is loaded!')

	elif checkpoint["num_classes"] ==2:
		labels_map = { # , is for saving data
					0: ',Noncontact,',
					1: ',Contact,',
				} 
		print('contact detection model is loaded!')
		
	return model.eval(), labels_map


# run tests

## modular LSTM

### run and test single modles

In [7]:

# Get the current working directory
main_path = os.getcwd()
num_features_lstm = 4
'''
#trained with all data
contact_detection_path= main_path +'/trainedModels/contactDetection/trainedModel_06_30_2023_10:16:53.pth'
collision_detection_path = main_path + '/trainedModels/collisionDetection/trainedModel_06_30_2023_09:07:24.pth'
localization_path = main_path + '/trainedModels_without_torque/localization/trainedModel_06_30_2023_09:08:08.pth'
'''
# trained with training set
contact_detection_path= main_path +'/trainedModels/contactDetection/trainedModel_01_24_2024_11:18:01.pth'
collision_detection_path = main_path + '/trainedModels/collisionDetection/trainedModel_01_24_2024_11:12:30.pth'
localization_path = main_path + '/trainedModels/localization/trainedModel_01_24_2024_11:15:06.pth'

window_length = 28
features_num = 28
dof = 7

# load model

model_contact, labels_map_contact = import_lstm_models(PATH=contact_detection_path, num_features_lstm=num_features_lstm)
model_collision, labels_map_collision = import_lstm_models(PATH=collision_detection_path, num_features_lstm=num_features_lstm)
model_localization, labels_map_localization = import_lstm_models(PATH=localization_path, num_features_lstm=num_features_lstm)
model_contact.double(); model_collision.double();model_localization.double()
#contact detection labels--> 0:contact, 1: collision
#contact localization labels--> 0:link5, 1:link6

num_classes= 2
confusionMatrix = ConfusionMatrix(task = "multiclass", num_classes= num_classes)

with torch.no_grad():
    collision= False; localization= False
    training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
    testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)
    y_pred, y_test = get_output(testing_data, model_contact)
    print("contact detection on the test set: \n",confusionMatrix(y_test , y_pred))
    class_report_test = classification_report(y_test, y_pred,digits=4)
    print(class_report_test)
    # test on training set
    #y_pred, y_train = get_output(training_data, model_contact)
    #print("on the train set: \n",confusionMatrix(y_train , y_pred))
   
    collision= True; localization= False
    training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
    testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)
    y_pred, y_test = get_output(testing_data, model_collision)
    print("collision detection on the test set: \n",confusionMatrix(y_test , y_pred))
    class_report_test = classification_report(y_test, y_pred,digits=4)
    print(class_report_test)
    # test on training set
    #y_pred, y_train = get_output(training_data, model_collision)
    #print("on the train set: \n",confusionMatrix(y_train , y_pred))
    
    collision= False; localization= True
    training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
    testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)
    y_pred, y_test = get_output(testing_data, model_localization)
    print("contact localization on the test set: \n",confusionMatrix(y_test , y_pred))
    class_report_test = classification_report(y_test, y_pred,digits=4)
    print(class_report_test)
    # test on training set
    #y_pred, y_train = get_output(training_data, model_localization)
    #print("on the train set: \n",confusionMatrix(y_train , y_pred))


contact detection model is loaded!
collision detection model is loaded!
localization model is loaded!
contact detection on the test set: 
 tensor([[360,   0],
        [  0, 302]])
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       360
           1     1.0000    1.0000    1.0000       302

    accuracy                         1.0000       662
   macro avg     1.0000    1.0000    1.0000       662
weighted avg     1.0000    1.0000    1.0000       662

collision detection on the test set: 
 tensor([[195,   4],
        [  9,  94]])
              precision    recall  f1-score   support

           0     0.9799    0.9559    0.9677       204
           1     0.9126    0.9592    0.9353        98

    accuracy                         0.9570       302
   macro avg     0.9463    0.9575    0.9515       302
weighted avg     0.9581    0.9570    0.9572       302

contact localization on the test set: 
 tensor([[157,   4],
        [  8, 133]])
     

### test modular design with 5 classes (similar to Mixed perception paper)

In [8]:
#contact detection labels--> 0:contact, 1: collision
#contact localization labels--> 0:link5, 1:link6

num_classes= 5
collision=False; localization = False
training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)

confusionMatrix = ConfusionMatrix(task = "multiclass", num_classes= num_classes)

model_contact.double(); model_collision.double();model_localization.double()

y_test = testing_data.data_target
y_pred = []
for i in range(testing_data.data_target.shape[0]):
    data_input = testing_data.__getitem__(i)[0].view(1,7,112)
    model_out = model_contact(data_input)
    #model_out = model_out.detach()
    contact = torch.argmax(model_out, dim=1).numpy()[0]
    if contact == 1:
        model_out = model_collision(data_input)
        model_out = model_out.detach()
        collision = torch.argmax(model_out, dim=1).numpy()[0]

        model_out = model_localization(data_input)
        model_out = model_out.detach()
        localization = torch.argmax(model_out, dim=1).numpy()[0]
        if collision==0:
            if localization ==0:
                y_pred.append(1)
            else:
                y_pred.append(2)
        else:
            if localization ==0:
                y_pred.append(3)
            else:
                y_pred.append(4)
            
    else:
        y_pred.append(0)
y_pred = torch.tensor(np.array(y_pred))
y_test = np.array(y_test).astype('int64')
class_report_test = classification_report(y_test, y_pred,digits=4)
print(class_report_test)


              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       360
           1     0.9608    0.8596    0.9074       114
           2     0.9072    0.9778    0.9412        90
           3     0.8305    0.9608    0.8909        51
           4     0.9773    0.9149    0.9451        47

    accuracy                         0.9637       662
   macro avg     0.9352    0.9426    0.9369       662
weighted avg     0.9660    0.9637    0.9638       662



### test modular design with 3 classes (similar to DML paper)

In [9]:
#contact detection labels--> 0:contact, 1: collision
#contact localization labels--> 0:link5, 1:link6

num_classes= 3
collision=False; localization = False
training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)

confusionMatrix = ConfusionMatrix(task = "multiclass", num_classes= num_classes)

model_contact.double(); model_collision.double();model_localization.double()

y_test = testing_data.data_target
y_pred = []
for i in range(testing_data.data_target.shape[0]):
    data_input = testing_data.__getitem__(i)[0].view(1,7,112)
    model_out = model_contact(data_input)
    #model_out = model_out.detach()
    contact = torch.argmax(model_out, dim=1).numpy()[0]
    if contact == 1:
        model_out = model_collision(data_input)
        model_out = model_out.detach()
        collision = torch.argmax(model_out, dim=1).numpy()[0]

        model_out = model_localization(data_input)
        model_out = model_out.detach()
        localization = torch.argmax(model_out, dim=1).numpy()[0]
        if collision==0:
            y_pred.append(1)
        else:
            y_pred.append(2)
    else:
        y_pred.append(0)
y_pred = torch.tensor(np.array(y_pred))
y_test = np.array(y_test).astype('int64')
class_report_test = classification_report(y_test, y_pred,digits=4)
print(class_report_test)


              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       360
           1     0.9799    0.9559    0.9677       204
           2     0.9126    0.9592    0.9353        98

    accuracy                         0.9804       662
   macro avg     0.9642    0.9717    0.9677       662
weighted avg     0.9809    0.9804    0.9805       662



## LSTM

### test model with 5 classes (similar to mixed_perception paper)

In [12]:
def import_models(PATH, num_class=5, network_type = 'main'):

	model = Sequence(num_class, network_type= network_type )
	checkpoint = torch.load(PATH)
	model.load_state_dict(checkpoint["model_state_dict"])
	model.eval()
	print('***  Models loaded  ***')
	return model.eval()

network_type = 'main'
num_classes= 5; collision= False; localization= False
model_collision.double()
training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)

model = import_models(os.path.split(main_path)[0]+'/AIModels/trainedModels/monolithicModels/trainedModel_05_17_2023_14:57:38.pth', num_class= num_classes, network_type= network_type) 

model.double()
with torch.no_grad():
	confusionMatrix = ConfusionMatrix(task = "multiclass", num_classes= num_classes)

	y_pred, y_test = get_output(testing_data, model)
	print("on the test set: \n",confusionMatrix(y_test , y_pred))


	class_report_test = classification_report(y_test, y_pred,digits=4)
	print(class_report_test)

	y_pred, y_train = get_output(training_data, model)
	print("on the train set: \n",confusionMatrix(y_train , y_pred))

***  Models loaded  ***
on the test set: 
 tensor([[360,   2,   0,   0,   0],
        [  0,  85,   2,   0,   0],
        [  0,  14,  86,   2,   2],
        [  0,  10,   0,  49,   1],
        [  0,   3,   2,   0,  44]])
              precision    recall  f1-score   support

           0     0.9945    1.0000    0.9972       360
           1     0.9770    0.7456    0.8458       114
           2     0.8269    0.9556    0.8866        90
           3     0.8167    0.9608    0.8829        51
           4     0.8980    0.9362    0.9167        47

    accuracy                         0.9426       662
   macro avg     0.9026    0.9196    0.9058       662
weighted avg     0.9481    0.9426    0.9416       662

on the train set: 
 tensor([[839,   0,   0,   0,   0],
        [  0, 202,   0,   0,   0],
        [  0,  17, 217,   0,   0],
        [  0,   5,   0, 129,   0],
        [  0,   1,   1,   0, 132]])


### test model with 5 classes (similar to DML paper)

In [14]:
def import_models(PATH, num_class=5, network_type = 'main'):

	model = Sequence(num_class, network_type= network_type )
	checkpoint = torch.load(PATH)
	model.load_state_dict(checkpoint["model_state_dict"])
	model.eval()
	print('***  Models loaded  ***')
	return model.eval()

network_type = 'main'
num_classes= 3; collision= False; localization= False
model_collision.double()
training_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_train.csv',num_classes=num_classes, collision=collision, localization= localization, num_features_lstm=num_features_lstm)
testing_data = create_tensor_dataset_without_torque(os.path.split(main_path)[0]+'/dataset/realData/contact_detection_test.csv',num_classes=num_classes, collision=collision, localization= localization,num_features_lstm=num_features_lstm)

model = import_models(os.path.split(main_path)[0]+'/AIModels/trainedModels/monolithicModels/trainedModel_05_20_2023_12:39:22.pth', num_class= num_classes, network_type= network_type) 

model.double()
with torch.no_grad():
	confusionMatrix = ConfusionMatrix(task = "multiclass", num_classes= num_classes)

	y_pred, y_test = get_output(testing_data, model)
	print("on the test set: \n",confusionMatrix(y_test , y_pred))

	class_report_test = classification_report(y_test, y_pred, digits=4)
	print(class_report_test)

	y_pred, y_train = get_output(training_data, model)
	print("on the train set: \n",confusionMatrix(y_train , y_pred))

***  Models loaded  ***
on the test set: 
 tensor([[360,   0,   0],
        [  0, 193,   9],
        [  0,  11,  89]])
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       360
           1     0.9554    0.9461    0.9507       204
           2     0.8900    0.9082    0.8990        98

    accuracy                         0.9698       662
   macro avg     0.9485    0.9514    0.9499       662
weighted avg     0.9700    0.9698    0.9699       662

on the train set: 
 tensor([[839,   0,   0],
        [  0, 441,   0],
        [  0,   2, 261]])


## Baselines

### DML

In [15]:
def classification_report_from_confusion_matrix(confusion_matrix):
    TP = confusion_matrix.diag()
    FP = confusion_matrix.sum(dim=0) - TP
    FN = confusion_matrix.sum(dim=1) - TP

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (precision * recall) / (precision + recall)

    class_report = {
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "support": TP + FN
    }

    return class_report


confusion_matrix= torch.tensor([[228, 0, 0],
                              [0, 102, 4],
                              [0, 2, 93],])

# Calculate class-wise metrics
class_report = classification_report_from_confusion_matrix(confusion_matrix)

# Print the formatted report
print("{:<15} {:<15} {:<15} {:<15}".format("Class", "Precision", "Recall", "F1-Score"))
print("-" * 60)
for i, (precision, recall, f1, support) in enumerate(zip(class_report["precision"], class_report["recall"], class_report["f1-score"], class_report["support"])):
    print("{:<15} {:<15.4f} {:<15.4f} {:<15.4f}".format(f"Class {i}", precision.item(), recall.item(), f1.item()))

# weighted-average
class_counts = np.sum(np.array(confusion_matrix), axis=1)
# Calculate weighted average precision
weights = class_counts / float(sum(class_counts))
micro_precision = np.sum(np.array(class_report['precision']) * weights)
micro_recall = np.sum(np.array(class_report['recall']) * weights)
micro_f1 = np.sum(np.array(class_report['f1-score']) * weights)

print("\nWeighted-average:")
print("{:<15} {:<15} {:<15}".format("Metric", "Score", ""))
print("-" * 30)
print("{:<15} {:<15.4f} {:<15}".format("Precision", micro_precision, ""))
print("{:<15} {:<15.4f} {:<15}".format("Recall", micro_recall, ""))
print("{:<15} {:<15.4f} {:<15}".format("F1-Score", micro_f1, ""))


# Calculate macro-average metrics
macro_precision = class_report["precision"].mean().item()
macro_recall = class_report["recall"].mean().item()
macro_f1 = class_report["f1-score"].mean().item()

# Print macro-average metrics
print("\nMacro-average:")
print("{:<15} {:<15} {:<15}".format("Metric", "Score", ""))
print("-" * 30)
print("{:<15} {:<15.4f} {:<15}".format("Precision", macro_precision, ""))
print("{:<15} {:<15.4f} {:<15}".format("Recall", macro_recall, ""))
print("{:<15} {:<15.4f} {:<15}".format("F1-Score", macro_f1, ""))

Class           Precision       Recall          F1-Score       
------------------------------------------------------------
Class 0         1.0000          1.0000          1.0000         
Class 1         0.9808          0.9623          0.9714         
Class 2         0.9588          0.9789          0.9687         

Weighted-average:
Metric          Score                          
------------------------------
Precision       0.9861                         
Recall          0.9860                         
F1-Score        0.9860                         

Macro-average:
Metric          Score                          
------------------------------
Precision       0.9798                         
Recall          0.9804                         
F1-Score        0.9801                         


### Mixed perception

In [16]:
import torch

def classification_report_from_confusion_matrix(confusion_matrix):
    TP = confusion_matrix.diag()
    FP = confusion_matrix.sum(dim=0) - TP
    FN = confusion_matrix.sum(dim=1) - TP

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (precision * recall) / (precision + recall)

    class_report = {
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "support": TP + FN
    }

    return class_report

# Your confusion matrix
confusion_matrix = torch.tensor([[242, 0, 3, 0, 1],
                               [0, 93, 4, 4, 1],
                               [0, 3, 83, 0, 0],
                               [0, 6, 0, 50, 0],
                               [0, 0, 2, 0, 52]])

# Calculate class-wise metrics
class_report = classification_report_from_confusion_matrix(confusion_matrix)

# Print class-wise metrics
print("{:<15} {:<15} {:<15} {:<15}".format("Class", "Precision", "Recall", "F1-Score"))
print("-" * 60)
for i, (precision, recall, f1, support) in enumerate(zip(class_report["precision"], class_report["recall"], class_report["f1-score"], class_report["support"])):
    print("{:<15} {:<15.4f} {:<15.4f} {:<15.4f}".format(f"Class {i}", precision.item(), recall.item(), f1.item()))

# Calculate macro-average metrics
macro_precision = class_report["precision"].mean().item()
macro_recall = class_report["recall"].mean().item()
macro_f1 = class_report["f1-score"].mean().item()

# weighted-average
class_counts = np.sum(np.array(confusion_matrix), axis=1)
# Calculate weighted average precision
weights = class_counts / float(sum(class_counts))
micro_precision = np.sum(np.array(class_report['precision']) * weights)
micro_recall = np.sum(np.array(class_report['recall']) * weights)
micro_f1 = np.sum(np.array(class_report['f1-score']) * weights)


print("\nWeighted-average:")
print("{:<15} {:<15} {:<15}".format("Metric", "Score", ""))
print("-" * 30)
print("{:<15} {:<15.4f} {:<15}".format("Precision", micro_precision, ""))
print("{:<15} {:<15.4f} {:<15}".format("Recall", micro_recall, ""))
print("{:<15} {:<15.4f} {:<15}".format("F1-Score", micro_f1, ""))

# Print macro-average metrics
print("\nMacro-average:")
print("{:<15} {:<15} {:<15}".format("Metric", "Score", ""))
print("-" * 30)
print("{:<15} {:<15.4f} {:<15}".format("Precision", macro_precision, ""))
print("{:<15} {:<15.4f} {:<15}".format("Recall", macro_recall, ""))
print("{:<15} {:<15.4f} {:<15}".format("F1-Score", macro_f1, ""))

Class           Precision       Recall          F1-Score       
------------------------------------------------------------
Class 0         1.0000          0.9837          0.9918         
Class 1         0.9118          0.9118          0.9118         
Class 2         0.9022          0.9651          0.9326         
Class 3         0.9259          0.8929          0.9091         
Class 4         0.9630          0.9630          0.9630         

Weighted-average:
Metric          Score                          
------------------------------
Precision       0.9567                         
Recall          0.9559                         
F1-Score        0.9561                         

Macro-average:
Metric          Score                          
------------------------------
Precision       0.9406                         
Recall          0.9433                         
F1-Score        0.9416                         
